# nbdev is getting a little %magic
> Introducing magic flags in nbdev.

- toc: true 
- badges: true
- comments: true
- categories: [nbdev, fastai, jupyter]
- author: Peter Butterfill

# Overview

nbdev is progressively getting magic commands that will eventually replace the special comments it currently uses, so you get autocomplete and documentation:

<img alt="images/nbdev-magic.gif" width="75%" align="left" src="images/nbdev-magic.gif" />

You'll also get feedback when flags are used incorrectly:

<img alt="images/nbdev-magic.gif"  width="75%" align="left" src="images/nbdev-magic-usage-error.png" />

# How do I start using magic flags?

Grab an [editable install of nbdev](https://github.com/fastai/nbdev/#installing) then run `nbdev_upgrade` from the command line - this will update notebooks that use comment flags like:

```python
#export special.module
```

to use magic flags:

```python
%nbdev_export special.module
```

To make magic flags work, `nbdev_upgrade` might need to add a new code cell to the top of the notebook:

```python
from nbdev import *
```

You can run `nbdev_upgrade` any number of times - which means you can update the same project every time new magic flags get added to nbdev.

## TODO

apart from `#hide` and `%nbdev_hide`, nbdev will ignore flags that are not in code cells - why?

test flags are created dynamically from settings.ini tst_flags

currently both comment and magic flags are supported and you can use both kinds in the same notebook

don't currently have a plan for deprecating comment flags

## Do magic flags work differently to comment flags?

Ideally, they would work the same but ... there is a difference when nbdev splits the source of a cell into flags and code;

If no magic flags are found, treat the first comment line as a flag

<img alt="split_flags_and_code example" width="450" align="left" src="images/split_flags_and_code.png" />

If magic flags are found, the flags part can contain multiple lines

<img alt="split_flags_and_code example" width="450" align="left" src="images/split_flags_and_code_magic.png" />

This could make a difference because nbdev writes just the code part to both the library and HTML docs.